# Logistic Regreesion（逻辑回归）

* 选用kernel learning_dl
~~~
conda activate learning_dl 

# conda remove --name learning_dl --all
~~~

In [16]:
import pandas as pd
import seaborn as sns
sns.set(context="notebook", style="whitegrid", palette="dark")
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

print("finish import module!!!")

finish import module!!!
